In [1]:
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import MultiTaskDataset
import evaluation

In [2]:
input_data = np.load('inputs.npy').astype(np.float32)
output_data = np.load('outputs.npy').astype(np.float32)

train_int, dev_int, train_out, dev_out = train_test_split(input_data, output_data, test_size=0.2, random_state=42)

train_out1 = train_out[:, 0, :]
train_out2 = train_out[:, 1, :]
train_out3 = train_out[:, 2, :]

dev_out1 = dev_out[:, 0, :]
dev_out2 = dev_out[:, 1, :]
dev_out3 = dev_out[:, 2, :]

In [3]:
print(np.mean(train_out1), np.min(train_out1), np.max(train_out1))
print(np.mean(train_out2), np.min(train_out2), np.max(train_out2))
print(np.mean(train_out3), np.min(train_out3), np.max(train_out3))

920.2192 -0.67927146 741954.56
0.0086741885 -0.3333267 0.6572305
0.004157325 -0.3333333 0.66662663


In [4]:
dm1 = np.load('Bolplanck_DM_xi.npy').astype(np.float32)
train_out1_dm = train_out1 / dm1
dev_out1_dm = dev_out1 / dm1

train_out2_dm_og = train_out2 / dm1
dev_out2_dm_og = dev_out2 / dm1

train_out3_dm_og = train_out3 / dm1
dev_out3_dm_og = dev_out3 / dm1

In [5]:
print(np.mean(train_out1_dm), np.min(train_out1_dm), np.max(train_out1_dm))
print(np.mean(train_out2_dm_og), np.min(train_out2_dm_og), np.max(train_out2_dm_og))
print(np.mean(train_out3_dm_og), np.min(train_out3_dm_og), np.max(train_out3_dm_og))

4.8700614 -1.815475 1162.3429
0.0017585435 -1.5572307 1.1663933
0.0003111336 -1.6395723 1.720901


In [6]:
indices = np.any(dev_out1_dm > 100, axis=1)
rows_to_delete_dev = np.where(indices)[0]
print(rows_to_delete_dev.shape)

(502,)


In [7]:
indices = np.any(train_out1_dm > 100, axis=1)
rows_to_delete_train = np.where(indices)[0]
print(rows_to_delete_train.shape)

(1881,)


In [8]:
train_out1_dm = np.delete(train_out1_dm, rows_to_delete_train, axis=0)
dev_out1_dm = np.delete(dev_out1_dm, rows_to_delete_dev, axis=0)

train_out2_dm_og = np.delete(train_out2_dm_og, rows_to_delete_train, axis=0)
dev_out2_dm_og = np.delete(dev_out2_dm_og, rows_to_delete_dev, axis=0)

train_out3_dm_og = np.delete(train_out3_dm_og, rows_to_delete_train, axis=0)
dev_out3_dm_og = np.delete(dev_out3_dm_og, rows_to_delete_dev, axis=0)

In [9]:
print(np.mean(train_out1_dm), np.min(train_out1_dm), np.max(train_out1_dm))
print(np.mean(train_out2_dm_og), np.min(train_out2_dm_og), np.max(train_out2_dm_og))
print(np.mean(train_out3_dm_og), np.min(train_out3_dm_og), np.max(train_out3_dm_og))

4.0426817 -0.019937249 99.98336
0.0016855038 -0.55319476 0.69765794
0.0002882932 -0.54974824 0.64470935


In [10]:
def column_wise_r(A, B):
    r = np.zeros(A.shape[1])
    for col in range(A.shape[1]):
        r[col] = np.mean(abs(A[:, col] / B[:, col]))
        r[col] = np.sqrt(r[col])
    return r

In [11]:
r12 = column_wise_r(train_out1_dm, train_out2_dm_og).astype(np.float32)
print(r12)
r13 = column_wise_r(train_out1_dm, train_out3_dm_og).astype(np.float32)
print(r13)

[897.1982   673.00494  679.99384  562.6633   420.83118  361.24155
 349.27252  317.8694   194.38054  127.01312   90.09065   89.07621
  50.156803  50.599228  49.930668  45.284874  45.746452  41.250507
  32.769413  27.539112]
[1074.1969   1043.1628    989.0476   1076.9342    840.737     807.39703
 1111.9567    998.92377   988.8078    540.1072    461.9973    373.55136
  232.99428   146.7894    202.56018   181.88087   145.0774    176.11647
  128.44656   113.359924]


In [14]:
train_out2_dm = train_out2_dm_og * r12 * 5
train_out3_dm = train_out3_dm_og * r13

In [15]:
print(np.mean(train_out2_dm), np.min(train_out2_dm), np.max(train_out2_dm))
print(np.mean(train_out3_dm), np.min(train_out3_dm), np.max(train_out3_dm))

0.454422 -86.09015 96.06441
0.06653836 -86.251595 96.081215


In [16]:
dev_out2_dm = dev_out2_dm_og * r12 * 5
dev_out3_dm = dev_out3_dm_og * r13

In [17]:
print(np.mean(dev_out2_dm), np.min(dev_out2_dm), np.max(dev_out2_dm))
print(np.mean(dev_out3_dm), np.min(dev_out3_dm), np.max(dev_out3_dm))

0.43712196 -66.888336 113.63487
0.06922737 -101.95363 74.44943


In [18]:
count1 = np.count_nonzero(train_out1_dm > 1)
count2 = np.count_nonzero(train_out2_dm > 1)
count3 = np.count_nonzero(train_out3_dm > 1)
print(count1, count2, count3)

1366128 463537 129577


In [19]:
count1 = np.count_nonzero(train_out1_dm < 0)
print(count1)

1


In [20]:
count1 = np.count_nonzero(train_out1_dm > 10)
count2 = np.count_nonzero(train_out2_dm > 10)
count3 = np.count_nonzero(train_out3_dm > 10)
print(count1, count2, count3)

163722 40659 9031


In [21]:
count1 = np.count_nonzero(train_out1_dm > 90)
count2 = np.count_nonzero(train_out2_dm > 90)
count3 = np.count_nonzero(train_out3_dm > 90)
print(count1, count2, count3)

364 3 1


In [23]:
train_int = np.delete(train_int, rows_to_delete_train, axis=0)
dev_int = np.delete(dev_int, rows_to_delete_dev, axis=0)

In [24]:
train_dataset = MultiTaskDataset.MultiTaskDataset(train_int, train_out1_dm, train_out2_dm, train_out3_dm)
train_dataloader = DataLoader(train_dataset, batch_size=128, shuffle=True)

dev_dataset = MultiTaskDataset.MultiTaskDataset(dev_int, dev_out1_dm, dev_out2_dm, dev_out3_dm)
dev_dataloader = DataLoader(dev_dataset)

In [25]:
class FCED(nn.Module):
    def __init__(self, input_dim=7, hidden_dim=128, bottleneck_dim=64, output_dim=20):
        super(FCED, self).__init__()
        
        self.encoder = nn.Sequential(
            nn.Linear(input_dim, hidden_dim),
            nn.BatchNorm1d(hidden_dim),
            nn.LeakyReLU(),
            nn.Linear(hidden_dim, hidden_dim),
            nn.BatchNorm1d(hidden_dim),
            nn.LeakyReLU(),
            nn.Linear(hidden_dim, hidden_dim),
            nn.BatchNorm1d(hidden_dim),
            nn.LeakyReLU(),
            nn.Linear(hidden_dim, hidden_dim),
            nn.BatchNorm1d(hidden_dim),
            nn.LeakyReLU(),
#             nn.Linear(hidden_dim, hidden_dim),
#             nn.BatchNorm1d(hidden_dim),
#             nn.LeakyReLU(),
#             nn.Linear(hidden_dim, hidden_dim),
#             nn.BatchNorm1d(hidden_dim),
#             nn.LeakyReLU(),
#             nn.Linear(hidden_dim, hidden_dim),
#             nn.BatchNorm1d(hidden_dim),
#             nn.LeakyReLU(),
#             nn.Linear(hidden_dim, hidden_dim),
#             nn.BatchNorm1d(hidden_dim),
#             nn.LeakyReLU(),
#             nn.Linear(hidden_dim, hidden_dim),
#             nn.BatchNorm1d(hidden_dim),
#             nn.LeakyReLU()
        )
        
        self.fc = nn.Sequential(
            nn.Linear(hidden_dim, hidden_dim),
            nn.ReLU(),
            nn.Linear(hidden_dim, bottleneck_dim),
            nn.ReLU(),
            nn.Linear(bottleneck_dim, bottleneck_dim),
            nn.ReLU(),
            nn.Linear(bottleneck_dim, bottleneck_dim),
            nn.ReLU(),
            nn.Linear(bottleneck_dim, output_dim)
        )
        
        self.decoder1 = nn.Sequential(
            nn.Conv1d(in_channels=1, out_channels=bottleneck_dim, kernel_size=3, stride=1, padding=1),
            nn.LeakyReLU(),
            nn.Conv1d(in_channels=bottleneck_dim, out_channels=bottleneck_dim, kernel_size=3, stride=1, padding=1),
            nn.LeakyReLU(),
#             nn.Conv1d(in_channels=bottleneck_dim, out_channels=bottleneck_dim, kernel_size=3, stride=1, padding=1),
#             nn.LeakyReLU(),
#             nn.Conv1d(in_channels=bottleneck_dim, out_channels=bottleneck_dim, kernel_size=3, stride=1, padding=1),
#             nn.LeakyReLU(),
            nn.Conv1d(in_channels=bottleneck_dim, out_channels=output_dim, kernel_size=3, stride=1, padding=1),
            nn.LeakyReLU(),
            nn.Flatten(),
            nn.Linear(output_dim*hidden_dim, output_dim)
        )
        
        self.decoder2 = nn.Sequential(
            nn.Conv1d(in_channels=1, out_channels=bottleneck_dim, kernel_size=3, stride=1, padding=1),
            nn.LeakyReLU(),
            nn.Conv1d(in_channels=bottleneck_dim, out_channels=bottleneck_dim, kernel_size=3, stride=1, padding=1),
            nn.LeakyReLU(),
            nn.Conv1d(in_channels=bottleneck_dim, out_channels=bottleneck_dim, kernel_size=3, stride=1, padding=1),
            nn.LeakyReLU(),
#             nn.Conv1d(in_channels=bottleneck_dim, out_channels=bottleneck_dim, kernel_size=3, stride=1, padding=1),
#             nn.LeakyReLU(),
            nn.Conv1d(in_channels=bottleneck_dim, out_channels=output_dim, kernel_size=3, stride=1, padding=1),
            nn.LeakyReLU(),
            nn.Flatten(),
            nn.Linear(output_dim*hidden_dim, output_dim)
        )
        
    def forward(self, x):
        encoded = self.encoder(x)
        task1 = self.fc(encoded)
        
        encoded = encoded.view(encoded.size(0), 1, -1)
        
        task2 = self.decoder1(encoded)
        task3 = self.decoder2(encoded)
        
        outputs = torch.stack((task1, task2, task3), dim=1)
        
        return outputs

In [29]:
def train_model(model, model_name, train_loader, dev_loader, num_epochs=2000, learning_rate=0.05, weight_decay=1e-4):
    criterion = nn.MSELoss()
    optimizer = optim.Adam(model.parameters(), lr=learning_rate, weight_decay=weight_decay)
    scheduler = optim.lr_scheduler.MultiStepLR(optimizer, milestones=np.arange(100, 1501, 100), gamma=0.5)
    
    best_model = None
    best_aapd = None
    
    train_losses = []
    val_losses = []
    val_vals = []

    for epoch in range(num_epochs):
        model.train()
        for batch in train_loader:
            input_data = batch['input_data']
            target1 = batch['out1']
            target2 = batch['out2']
            target3 = batch['out3']
            
            optimizer.zero_grad()
            outputs = model(input_data)
            #print(outputs.shape)
            train_loss1 = criterion(outputs[:, 0, :], target1)
            train_loss2 = criterion(outputs[:, 1, :], target2)
            train_loss3 = criterion(outputs[:, 2, :], target3)
            
            train_loss = train_loss1 + train_loss2 + train_loss3
            
            train_loss.backward()
            optimizer.step()
        scheduler.step()
        train_losses.append(train_loss.item())
        
#         model.eval()
        
#         total_aapd = 0
#         num_samples = 0
        
#         with torch.no_grad():
#             for batch in dev_loader:
#                 dev_int = batch['input_data']
#                 dev_out1 = batch['out1']
#                 dev_out2 = batch['out2']
#                 dev_out3 = batch['out3']
                
#                 predictions = model(dev_int)
                
#                 val_loss1 = criterion(predictions[:, 0, :], dev_out1)
#                 batch_aapd1 = AAPD(predictions[:, 0, :], dev_out1)
                
#                 val_loss2 = criterion(predictions[:, 1, :], dev_out2)
#                 batch_aapd2 = AAPD(predictions[:, 1, :], dev_out2)
                
#                 val_loss3 = criterion(predictions[:, 2, :], dev_out3)
#                 batch_aapd3 = AAPD(predictions[:, 2, :], dev_out3)
                
#                 val_loss = val_loss1 + val_loss2 + val_loss3
#                 batch_aapd = (batch_aapd1 + batch_aapd2 + batch_aapd3) / 3
                    
#                 total_aapd += batch_aapd * len(dev_int)
#                 num_samples += len(dev_int)
            
#             val_losses.append(val_loss.item())
#             aapd = total_aapd / num_samples
#             val_vals.append(aapd)
            
#             if best_aapd == None or aapd < best_aapd:
#                 best_aapd = aapd
#                 best_model = model
#                 torch.save(model.state_dict(), model_name)
        
        print(f'Epoch [{epoch + 1}/{num_epochs}] - Loss: {train_loss.item():.4f}')#; AAPD: {aapd}')
        
    plt.plot(np.arange(num_epochs), train_losses, '.', label = 'train set')
    #plt.plot(np.arange(num_epochs), val_losses, 'x', label = 'val set')
    plt.legend(loc='upper right')
    plt.xlabel('num epochs')
    plt.ylabel('loss')
    plt.xscale('log')
    plt.yscale('log')
    #plt.save('loss.pdf')
    plt.show()
    
#     plt.plot(np.arange(len(val_vals)), val_vals)
#     plt.xlabel('num epochs')
#     plt.ylabel('<|$\hat{y}$ - y| / y>')
#     plt.xscale('log')
#     plt.yscale('log')
#     #plt.save('validation.pdf')
#     plt.show()

#    return best_model, train_losses, val_losses, val_vals

In [ ]:
model = FCED()
best_model_mul_FCED = train_model(model, 'best_model_mul_FCED.pth', train_dataloader, dev_dataloader)

Epoch [1/2000] - Loss: 4382.5576
Epoch [2/2000] - Loss: 119.3752
Epoch [3/2000] - Loss: 71.9488
Epoch [4/2000] - Loss: 242.1225
Epoch [5/2000] - Loss: 291.6582
Epoch [6/2000] - Loss: 59.7375
Epoch [7/2000] - Loss: 137.5274
Epoch [8/2000] - Loss: 428.1820
Epoch [9/2000] - Loss: 3711711.0000
Epoch [10/2000] - Loss: 181368624.0000
Epoch [11/2000] - Loss: 4426292.5000
Epoch [12/2000] - Loss: 33116016.0000
Epoch [13/2000] - Loss: 39356732.0000
Epoch [14/2000] - Loss: 4760542.5000
Epoch [15/2000] - Loss: 10593319.0000
Epoch [16/2000] - Loss: 422877.1250
Epoch [17/2000] - Loss: 7506933.5000
Epoch [18/2000] - Loss: 10832909.0000
Epoch [19/2000] - Loss: 4932703.0000
Epoch [20/2000] - Loss: 15306989.0000
Epoch [21/2000] - Loss: 473676.1562
Epoch [22/2000] - Loss: 22757346.0000
Epoch [23/2000] - Loss: 36883.3281
Epoch [24/2000] - Loss: 229739.0781
Epoch [25/2000] - Loss: 821671.7500
Epoch [26/2000] - Loss: 719777.8750
Epoch [27/2000] - Loss: 500510.4062
Epoch [28/2000] - Loss: 153968.7188
Epoch [

Epoch [232/2000] - Loss: 199.5658
Epoch [233/2000] - Loss: 1372.4590
Epoch [234/2000] - Loss: 249.4496
Epoch [235/2000] - Loss: 399.6759
Epoch [236/2000] - Loss: 397.0748
Epoch [237/2000] - Loss: 4426.8799
Epoch [238/2000] - Loss: 577.8541
Epoch [239/2000] - Loss: 154.7923
Epoch [240/2000] - Loss: 649.4290
Epoch [241/2000] - Loss: 8539.9160
Epoch [242/2000] - Loss: 256.0403
Epoch [243/2000] - Loss: 101.4964
Epoch [244/2000] - Loss: 82.0552
Epoch [245/2000] - Loss: 59.1421
Epoch [246/2000] - Loss: 91.6049
Epoch [247/2000] - Loss: 65.7223
Epoch [248/2000] - Loss: 58.9507
Epoch [249/2000] - Loss: 1147.9728
Epoch [250/2000] - Loss: 103.5647
Epoch [251/2000] - Loss: 46.5411
Epoch [252/2000] - Loss: 223.1834
Epoch [253/2000] - Loss: 81.7308
Epoch [254/2000] - Loss: 62.1815
Epoch [255/2000] - Loss: 54.3347
Epoch [256/2000] - Loss: 44.3536
Epoch [257/2000] - Loss: 45.4262
Epoch [258/2000] - Loss: 56.5363
Epoch [259/2000] - Loss: 57.0573
Epoch [260/2000] - Loss: 36.5199
Epoch [261/2000] - Loss:

Epoch [479/2000] - Loss: 10.6408
Epoch [480/2000] - Loss: 22.8915
Epoch [481/2000] - Loss: 16.6214
Epoch [482/2000] - Loss: 19.9986
Epoch [483/2000] - Loss: 16.0804
Epoch [484/2000] - Loss: 33.5965
Epoch [485/2000] - Loss: 22.4754
Epoch [486/2000] - Loss: 10.0337
Epoch [487/2000] - Loss: 16.2638
Epoch [488/2000] - Loss: 11.6357
Epoch [489/2000] - Loss: 25.0502
Epoch [490/2000] - Loss: 16.3561
Epoch [491/2000] - Loss: 13.0063
Epoch [492/2000] - Loss: 16.9706
Epoch [493/2000] - Loss: 12.5100
Epoch [494/2000] - Loss: 15.3275
Epoch [495/2000] - Loss: 7.8989
Epoch [496/2000] - Loss: 11.9215
Epoch [497/2000] - Loss: 12.2323
Epoch [498/2000] - Loss: 16.5133
Epoch [499/2000] - Loss: 26.9326
Epoch [500/2000] - Loss: 10.0406
Epoch [501/2000] - Loss: 9.4683
Epoch [502/2000] - Loss: 13.7105
Epoch [503/2000] - Loss: 8.1181
Epoch [504/2000] - Loss: 24.2997
Epoch [505/2000] - Loss: 12.0966
Epoch [506/2000] - Loss: 9.4271
Epoch [507/2000] - Loss: 12.5209
Epoch [508/2000] - Loss: 14.5035
Epoch [509/200

Epoch [730/2000] - Loss: 24.7140
Epoch [731/2000] - Loss: 10.2510
Epoch [732/2000] - Loss: 12.7323
Epoch [733/2000] - Loss: 15.2136
Epoch [734/2000] - Loss: 8.6889
Epoch [735/2000] - Loss: 6.9659
Epoch [736/2000] - Loss: 19.6218
Epoch [737/2000] - Loss: 15.2367
Epoch [738/2000] - Loss: 6.1884
Epoch [739/2000] - Loss: 10.1300
Epoch [740/2000] - Loss: 16.1489
Epoch [741/2000] - Loss: 9.8726
Epoch [742/2000] - Loss: 15.3884
Epoch [743/2000] - Loss: 6.9382
Epoch [744/2000] - Loss: 17.2096
Epoch [745/2000] - Loss: 27.2027
Epoch [746/2000] - Loss: 19.3757
Epoch [747/2000] - Loss: 8.9620
Epoch [748/2000] - Loss: 16.7317
Epoch [749/2000] - Loss: 11.3506
Epoch [750/2000] - Loss: 14.7881
Epoch [751/2000] - Loss: 12.7299
Epoch [752/2000] - Loss: 15.2689
Epoch [753/2000] - Loss: 15.3609
Epoch [754/2000] - Loss: 9.1584
Epoch [755/2000] - Loss: 13.4644
Epoch [756/2000] - Loss: 21.9752
Epoch [757/2000] - Loss: 11.5290
Epoch [758/2000] - Loss: 14.9610
Epoch [759/2000] - Loss: 22.2672
Epoch [760/2000] 

Epoch [981/2000] - Loss: 8.3759
Epoch [982/2000] - Loss: 17.6457
Epoch [983/2000] - Loss: 13.3289
Epoch [984/2000] - Loss: 23.1266
Epoch [985/2000] - Loss: 11.6420
Epoch [986/2000] - Loss: 18.1823
Epoch [987/2000] - Loss: 7.4028
Epoch [988/2000] - Loss: 6.0129
Epoch [989/2000] - Loss: 12.1236
Epoch [990/2000] - Loss: 4.5403
Epoch [991/2000] - Loss: 5.0376
Epoch [992/2000] - Loss: 14.4365
Epoch [993/2000] - Loss: 14.6672
Epoch [994/2000] - Loss: 12.4372
Epoch [995/2000] - Loss: 10.9489
Epoch [996/2000] - Loss: 16.5549
Epoch [997/2000] - Loss: 24.2046
Epoch [998/2000] - Loss: 13.9351
Epoch [999/2000] - Loss: 9.2462
Epoch [1000/2000] - Loss: 8.7803
Epoch [1001/2000] - Loss: 11.2086
Epoch [1002/2000] - Loss: 9.7233
Epoch [1003/2000] - Loss: 11.9198
Epoch [1004/2000] - Loss: 7.5626
Epoch [1005/2000] - Loss: 14.8271
Epoch [1006/2000] - Loss: 27.7920
Epoch [1007/2000] - Loss: 9.2679
Epoch [1008/2000] - Loss: 13.1936
Epoch [1009/2000] - Loss: 11.3213
Epoch [1010/2000] - Loss: 8.8933
Epoch [101

Epoch [1225/2000] - Loss: 14.7301
Epoch [1226/2000] - Loss: 9.9523
Epoch [1227/2000] - Loss: 16.0778
Epoch [1228/2000] - Loss: 8.5693
Epoch [1229/2000] - Loss: 9.2111
Epoch [1230/2000] - Loss: 14.8168
Epoch [1231/2000] - Loss: 15.8418
Epoch [1232/2000] - Loss: 10.8267
Epoch [1233/2000] - Loss: 8.2194
Epoch [1234/2000] - Loss: 18.8516
Epoch [1235/2000] - Loss: 10.7719
Epoch [1236/2000] - Loss: 6.6898
Epoch [1237/2000] - Loss: 7.3984
Epoch [1238/2000] - Loss: 8.9598
Epoch [1239/2000] - Loss: 11.7616
Epoch [1240/2000] - Loss: 12.4944
Epoch [1241/2000] - Loss: 17.1273
Epoch [1242/2000] - Loss: 6.7077
Epoch [1243/2000] - Loss: 14.3592
Epoch [1244/2000] - Loss: 10.3947
Epoch [1245/2000] - Loss: 12.8369
Epoch [1246/2000] - Loss: 10.5681
Epoch [1247/2000] - Loss: 18.3927
Epoch [1248/2000] - Loss: 10.1369
Epoch [1249/2000] - Loss: 14.8279
Epoch [1250/2000] - Loss: 10.3426
Epoch [1251/2000] - Loss: 14.0149
Epoch [1252/2000] - Loss: 19.9177
Epoch [1253/2000] - Loss: 15.5983
Epoch [1254/2000] - Lo

Epoch [1469/2000] - Loss: 21.9164
Epoch [1470/2000] - Loss: 9.6855
Epoch [1471/2000] - Loss: 12.1008
Epoch [1472/2000] - Loss: 6.2301
Epoch [1473/2000] - Loss: 13.7042
Epoch [1474/2000] - Loss: 16.7507
Epoch [1475/2000] - Loss: 23.0378
Epoch [1476/2000] - Loss: 11.8033
Epoch [1477/2000] - Loss: 13.1677
Epoch [1478/2000] - Loss: 8.2684
Epoch [1479/2000] - Loss: 13.1013
Epoch [1480/2000] - Loss: 12.8199
Epoch [1481/2000] - Loss: 12.9140
Epoch [1482/2000] - Loss: 9.8867
Epoch [1483/2000] - Loss: 16.0153
Epoch [1484/2000] - Loss: 10.3566
Epoch [1485/2000] - Loss: 6.7386
Epoch [1486/2000] - Loss: 18.1375
Epoch [1487/2000] - Loss: 10.9039
Epoch [1488/2000] - Loss: 10.2991
Epoch [1489/2000] - Loss: 8.1722
Epoch [1490/2000] - Loss: 16.8222
Epoch [1491/2000] - Loss: 10.3694
Epoch [1492/2000] - Loss: 15.3442
Epoch [1493/2000] - Loss: 12.0899
Epoch [1494/2000] - Loss: 4.0117
Epoch [1495/2000] - Loss: 14.1113
Epoch [1496/2000] - Loss: 6.5852
Epoch [1497/2000] - Loss: 21.7629
Epoch [1498/2000] - Lo

Epoch [1713/2000] - Loss: 9.4572
Epoch [1714/2000] - Loss: 17.9286
Epoch [1715/2000] - Loss: 10.8003
Epoch [1716/2000] - Loss: 8.4692
Epoch [1717/2000] - Loss: 12.7139
Epoch [1718/2000] - Loss: 18.3066
Epoch [1719/2000] - Loss: 11.7509
Epoch [1720/2000] - Loss: 12.3635
Epoch [1721/2000] - Loss: 14.1722
Epoch [1722/2000] - Loss: 6.7340
Epoch [1723/2000] - Loss: 9.7884
Epoch [1724/2000] - Loss: 7.6497
Epoch [1725/2000] - Loss: 11.6019
Epoch [1726/2000] - Loss: 10.3473
Epoch [1727/2000] - Loss: 8.9465
Epoch [1728/2000] - Loss: 15.5096
Epoch [1729/2000] - Loss: 9.9644
Epoch [1730/2000] - Loss: 12.4620
Epoch [1731/2000] - Loss: 13.2724
Epoch [1732/2000] - Loss: 11.8421
Epoch [1733/2000] - Loss: 13.8164
Epoch [1734/2000] - Loss: 21.7927
Epoch [1735/2000] - Loss: 7.9797
Epoch [1736/2000] - Loss: 16.7074
Epoch [1737/2000] - Loss: 13.3548
Epoch [1738/2000] - Loss: 18.1760
Epoch [1739/2000] - Loss: 21.5213
Epoch [1740/2000] - Loss: 10.0617
Epoch [1741/2000] - Loss: 12.1873
Epoch [1742/2000] - Lo